In [1]:
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from pymystem3 import Mystem
import re
from collections import Counter

In [2]:
data = pd.read_csv('C:/Users/atche/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Сведения об используемой памяти

In [3]:
data.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 11.4 MB


In [4]:
for dtype in ['float64','int64','object']:
    selected_dtype = data.select_dtypes(include=[dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b / 1024 ** 2
    print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb))

Average memory usage for float64 columns: 0.11 MB
Average memory usage for int64 columns: 0.14 MB
Average memory usage for object columns: 1.70 MB


# Замена пропущенных значений

Определим количество пропущенных значений по столбцам

In [5]:
data.isna().sum().to_frame(name = "nans").query("nans > 0")["nans"]

days_employed    2174
total_income     2174
Name: nans, dtype: int64

Заменим пропущенные значения в признаке total_income - ежемесячный доход. Заменим пропущенные значения медианным значением

In [6]:
data = data.sort_values(by = 'total_income', ascending = True)

In [7]:
data = data.fillna({'total_income': data.total_income.median()})
data = data.sort_index(ascending = True)
data.isna().sum().to_frame(name = "nans").query("nans > 0")["nans"]

days_employed    2174
Name: nans, dtype: int64

Проверим, сколько строк в датафрейме имею корректное значение в признаке days_employed. Примем значение корректным находящееся в диапазоне от 0 до 30000 дней.

In [8]:
((data.days_employed > 0) & (data.days_employed < 30000)).value_counts()

False    21525
Name: days_employed, dtype: int64

Видим, что данное условие не выполняется ни для одной строки датафрейма. Чтобы избежать ошибочной трактовки и в связи с тем, что в параметре days_employed нет необходимости для выполнения задания, удалим данный параметр из датафрейма.

In [9]:
del data['days_employed']

# Преобразование типов данных

С целью сокращения потребления памяти и повышения удобства анализа заменим вещественный тип данных на целочисленный в параметре total_income

In [10]:
data['total_income'] = data['total_income'].astype(np.int64)

# Удаление дубликатов

В датафрейме было найдено 54 дубликата. Все они были удалены с помощью функции drop_duplicates(). 

Дубликаты могут появляться из-за ошибок ввода. 

In [11]:
data.duplicated().sum()

54

In [12]:
data[data.duplicated()]

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,41,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для семьи
4182,1,34,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,145017,свадьба
4851,0,60,среднее,1,гражданский брак,1,F,пенсионер,0,145017,свадьба
5557,0,58,среднее,1,гражданский брак,1,F,пенсионер,0,145017,сыграть свадьбу
7808,0,57,среднее,1,гражданский брак,1,F,пенсионер,0,145017,на проведение свадьбы
8583,0,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,145017,дополнительное образование
9238,2,34,среднее,1,женат / замужем,0,F,сотрудник,0,145017,покупка жилья для сдачи
9528,0,66,среднее,1,вдовец / вдова,2,F,пенсионер,0,145017,операции со своей недвижимостью
9627,0,56,среднее,1,женат / замужем,0,F,пенсионер,0,145017,операции со своей недвижимостью
10462,0,62,среднее,1,женат / замужем,0,F,пенсионер,0,145017,покупка коммерческой недвижимости


In [13]:
data = data.drop_duplicates()

In [14]:
data

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


In [15]:
data = data.reset_index()

In [16]:
del data['index']

In [17]:
data

,children,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья
1,1,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля
2,0,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья
3,3,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование
4,0,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу
5,0,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья
6,0,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем
7,0,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823,образование
8,2,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы
9,0,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи


# Категоризация данных

Необходимо разделить людей на категории по возрасту.

Категоризация для признака "dob_years": 
0 - 18-24 года (период, который большинство посвящают получению образования);
1 - 25-35 лет (период создания семей);
2 - 36-50 лет (период воспитания и содержания детей);
3 - >51 года (период активных трат на здоровье).

Пропущенным значениям, заполненным, как 0, придадим категорию "-1". Впоследствии обработаем их и заполним.

In [18]:
a = data['dob_years']
a = np.where(a <= 17, -1, a)
a = np.where((a > 17) & (a < 25), 0, a)
a = np.where((a > 24) & (a < 36), 1, a)
a = np.where((a > 35) & (a < 51), 2, a)
a = np.where(a > 50, 3, a)
data = data.assign(dob_years = a)
data['dob_years'].value_counts()

 2    8130
 3    6655
 1    5709
 0     876
-1     101
Name: dob_years, dtype: int64

Колонки "education" и "family_status" предлагается удалить, так как колонки "education_id", "family_status_id" несут ту же информацию. 

Для того, чтобы была возможность понять смысл колонок "education_id", "family_status_id", составляются словари.

Для "education_id":
0 - высшее;
1 - среднее;
2 - неоконченное высшее;
3 - начальное;
4 - ученая степень.

Для "family_status_id":
0 - женат / замужем;
1 - гражданский брак;
2 - вдовец / вдова;
3 - в разводе;
4 - не женат / не замужем.

Для "income_type" некоторые категории необходимо объединить, так как "студент" является аналогом "безработный", "предприниматель" - аналог "компаньон", "в декрете" - аналог "сотрудник", заменить на числовые значения, так как тип int занимает меньше памяти по сравнению с object, и с данным типом данных проще работать при анализе.

Словарь для "income_type":
0 - сотрудник;
1 - компаньон;
2 - пенсионер;
3 - госслужащий;
4 - безработный.

In [19]:
d_1 = data.drop(['education', 'family_status'], axis = 1)

In [20]:
a = d_1['income_type'].replace('предприниматель', 'компаньон') \
    .replace('в декрете', 'сотрудник') \
    .replace('студент', 'безработный')
d_1 = d_1.assign(income_type = a)
d_1['income_type'].value_counts()

сотрудник      11092
компаньон       5082
пенсионер       3837
госслужащий     1457
безработный        3
Name: income_type, dtype: int64

In [21]:
dict_1 = {'сотрудник': 0, 'компаньон': 1, 'пенсионер': 2, 'госслужащий': 3, 'безработный': 4}
d_1 = d_1.replace({"income_type": dict_1})
d_1['income_type'].value_counts()

0    11092
1     5082
2     3837
3     1457
4        3
Name: income_type, dtype: int64

Значение признака "gender" также заменим числовыми: 0 - female; 1 - male. XNA заменим, как -1, так как ошибочное значение. Впоследствии, необходимо будет определить к какой категории ближе данный элемент.

In [22]:
d_1['gender'].value_counts()

F      14189
M       7281
XNA        1
Name: gender, dtype: int64

In [23]:
d_1['gender'] = d_1['gender'].replace('F', 0) \
    .replace('M', 1) \
    .replace('XNA', -1)

Необходимо категоризовать признак (total_income). Для этого разбили значения на перцентили. Необходимо будет проверить, на сколько такие категории являются показательными

Словарь:
0 - <=98554;
1 - 98554-135514;
2 - 135514-156400;
3 - 156400-214546;
4 - > 214546.

In [24]:
d_2 = d_1.copy(deep=True)
d_2.head()

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,2,0,0,0,0,0,253875,покупка жилья
1,1,2,1,0,0,0,0,112080,приобретение автомобиля
2,0,1,1,0,1,0,0,145885,покупка жилья
3,3,1,1,0,1,0,0,267628,дополнительное образование
4,0,3,1,1,0,2,0,158616,сыграть свадьбу


In [25]:
d_2 = d_2.query("total_income != 0")
np.percentile(d_2['total_income'], q = [20, 40, 60, 80])

array([ 98554., 135514., 156400., 214546.])

In [26]:
n = d_1['total_income']
n = np.where((n > 0) & (n < 98554), 0, n)
n = np.where((n >= 98554) & (n < 135514), 1, n)
n = np.where((n >= 135514) & (n < 156400), 2, n)
n = np.where((n >= 156400) & (n < 214546), 3, n)
n = np.where(n >= 214546, 4, n)
d_1 = d_1.assign(total_income = n)
d_1['total_income'].value_counts()

4    4295
3    4294
2    4294
1    4294
0    4294
Name: total_income, dtype: int64

Для того, чтобы категоризовать признак "purpose", необходимо провести лемматизацию и выделить основные категории.

In [27]:
d_1['purpose'].value_counts()

свадьба                                   793
на проведение свадьбы                     773
сыграть свадьбу                           769
операции с недвижимостью                  675
покупка коммерческой недвижимости         662
операции с жильем                         652
покупка жилья для сдачи                   652
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          625
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

In [28]:
m = Mystem()

Находим леммы среди целей получения кредита

In [29]:
purpose_list = d_1['purpose']
purpose_list = purpose_list.to_string()
purpose_list = str(purpose_list)
for i in range(21470): 
    purpose_list = purpose_list.replace('\n' + str(i + 1) + ' ', '')
purpose_list = re.sub(r'\s+', ' ', purpose_list)
purpose_list = purpose_list.replace('0 ', '') \
    .replace(' Name: purpose, Length: 21471, dtype: object', '')
lemm = m.lemmatize(purpose_list)

Выводим леммы и частоту их встречания, удаляем несодержательные леммы

In [30]:
lemm = [a for a in lemm if a != ' ']
lemm = [a for a in lemm if a != 'покупка']
lemm = [a for a in lemm if a != 'приобретение'] 
lemm = [a for a in lemm if a != 'дополнительный']
lemm = [a for a in lemm if a != 'сыграть']
lemm = [a for a in lemm if a != 'с']
lemm = [a for a in lemm if a != 'для'] 
lemm = [a for a in lemm if a != 'коммерческий']
lemm = [a for a in lemm if a != 'собственный']
lemm = [a for a in lemm if a != 'подержать']
lemm = [a for a in lemm if a != 'свой']
lemm = [a for a in lemm if a != 'со'] 
lemm = [a for a in lemm if a != 'заниматься']
lemm = [a for a in lemm if a != 'получение']
lemm = [a for a in lemm if a != 'высокий']
lemm = [a for a in lemm if a != 'подержанный'] 
lemm = [a for a in lemm if a != 'профильный']
lemm = [a for a in lemm if a != 'сдача'] 
lemm = [a for a in lemm if a != '\n']
lemm = [a for a in lemm if a != 'на']
lemm = [a for a in lemm if a != 'проведение']
lemm = [a for a in lemm if a != 'жилой']

counter = Counter(lemm)
counter

Counter({'жилье': 4461,
         'автомобиль': 4308,
         'образование': 4014,
         'свадьба': 2335,
         'операция': 2604,
         'семья': 638,
         'недвижимость': 6353,
         'строительство': 1879,
         'сделка': 941,
         'ремонт': 607})

Видим, что основными целями являются "недвижимость", "автомобиль", "образование", "свадьба". Необходимо проверить оставшиеся леммы, в каком значении они употребялются

Леммы "операция", "семья", "ремонт" относятся к категории "недвижимость". Лемма "сделка" относится к категории "автомобиль".

In [31]:
m = d_1[d_1['purpose'].str.contains("операц")]
m.purpose.unique()

array(['операции с жильем', 'операции с коммерческой недвижимостью',
       'операции со своей недвижимостью', 'операции с недвижимостью'],
      dtype=object)

In [32]:
m = d_1[d_1['purpose'].str.contains("семь")]
m.purpose.unique()

array(['покупка жилья для семьи'], dtype=object)

In [33]:
m = d_1[d_1['purpose'].str.contains("сделк")]
m.purpose.unique()

array(['сделка с подержанным автомобилем', 'сделка с автомобилем'],
      dtype=object)

In [34]:
m = d_1[d_1['purpose'].str.contains("ремонт")]
m.purpose.unique()

array(['ремонт жилью'], dtype=object)

Заменим столбец "purpose" леммами, которые будут являться категориями целей получения кредита.

In [35]:
d_1.loc[d_1['purpose'].str.contains("авто"), 'purpose'] = 'автомобиль'
d_1.loc[d_1['purpose'].str.contains("жил"), 'purpose'] = 'недвижимость'
d_1.loc[d_1['purpose'].str.contains("недвиж"), 'purpose'] = 'недвижимость'
d_1.loc[d_1['purpose'].str.contains("образов"), 'purpose'] = 'образование'
d_1.loc[d_1['purpose'].str.contains("свад"), 'purpose'] = 'свадьба'

In [36]:
d_1.purpose.unique()

array(['недвижимость', 'автомобиль', 'образование', 'свадьба'],
      dtype=object)

Заменим категориии 'недвижимость', 'автомобиль', 'образование', 'свадьба' на '0', '1', '2', '3'. 

In [37]:
dict_3 = {'недвижимость': 0, 'автомобиль': 1, 'образование': 2, 'свадьба': 3}
d_1 = d_1.replace({"purpose": dict_3})

In [38]:
d_1

,children,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,2,0,0,0,0,0,4,0
1,1,2,1,0,0,0,0,1,1
2,0,1,1,0,1,0,0,2,0
3,3,1,1,0,1,0,0,4,2
4,0,3,1,1,0,2,0,3,3
5,0,1,0,1,1,1,0,4,0
6,0,2,0,0,0,1,0,4,0
7,0,2,1,0,1,0,0,2,2
8,2,1,0,1,0,0,0,0,3
9,0,2,1,0,1,0,0,2,0


# Работа с некорректными значениями

Значения "-1" и "20" в признаке children являются некорректными. Их стоит заменить на ошибочную категорию "-1"

In [39]:
d_1['children'].value_counts()

 0     14107
 1      4809
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

In [40]:
d = d_1['children'].replace(20, -1)
d_1 = d_1.assign(children = d)
d_1['children'].value_counts()

 0    14107
 1     4809
 2     2052
 3      330
-1      123
 4       41
 5        9
Name: children, dtype: int64

# Поиск зависимостей

Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [62]:
t_1 = d_1.groupby(['children'], as_index = False).aggregate({'debt': 'count'}) \
    .rename(columns = {'debt': 'total'})
t_1

,children,total
0,-1,123
1,0,14107
2,1,4809
3,2,2052
4,3,330
5,4,41
6,5,9


In [63]:
t_2 = d_1.groupby(['children'], as_index = False).aggregate({'debt': 'sum'}) \
    .rename(columns = {'debt': 'amount_of_debts'})
t_2['total'] = t_1['total']
t_2['percent'] = t_2['amount_of_debts'] / t_2['total'] * 100
t_2

,children,amount_of_debts,total,percent
0,-1,9,123,7.317073
1,0,1063,14107,7.535266
2,1,444,4809,9.232689
3,2,194,2052,9.454191
4,3,27,330,8.181818
5,4,4,41,9.756098
6,5,0,9,0.000000


На данной сводной таблице  видно, что зависимости между наличем детей и возвартом кредита в срок нет

Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [64]:
t_3 = d_1.groupby(['family_status_id'], as_index = False).aggregate({'debt': 'count'}) \
    .rename(columns = {'debt': 'total'})
t_3

,family_status_id,total
0,0,12344
1,1,4163
2,2,959
3,3,1195
4,4,2810


In [65]:
t_4 = d_1.groupby(['family_status_id'], as_index = False).aggregate({'debt': 'sum'}) \
    .rename(columns = {'debt': 'amount_of_debts'})
t_4['total'] = t_3['total']
t_4['percent'] = t_4['amount_of_debts'] / t_4['total'] * 100
t_4

,family_status_id,amount_of_debts,total,percent
0,0,931,12344,7.542126
1,1,388,4163,9.320202
2,2,63,959,6.569343
3,3,85,1195,7.112971
4,4,274,2810,9.750890


На данной сводной таблице  видно, что зависимости между семейным статусом и возвартом кредита в срок нет

Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [66]:
t_5 = d_1.groupby(['total_income'], as_index = False).aggregate({'debt': 'count'}) \
    .rename(columns = {'debt': 'total'})
t_5

,total_income,total
0,0,4294
1,1,4294
2,2,4294
3,3,4294
4,4,4295


In [61]:
t_6 = d_1.groupby(['total_income'], as_index = False).aggregate({'debt': 'sum'}) \
    .rename(columns = {'debt': 'amount_of_debts'})
t_6['total'] = t_5['total']
t_6['percent'] = t_6['amount_of_debts'] / t_6['total'] * 100
t_6

,total_income,amount_of_debts,total,percent
0,0,344,4294,8.011178
1,1,355,4294,8.267350
2,2,370,4294,8.616674
3,3,372,4294,8.663251
4,4,300,4295,6.984866


При доходе свыше 214000 рублей задолженностей кредита меньше всего. Но стоит ли считать разницу в 1,5% показательной?

Как разные цели кредита влияют на его возврат в срок?

In [71]:
t_7 = d_1.groupby(['purpose']).aggregate({'debt': 'count'}) \
    .rename(columns = {'debt': 'total'})
t_7

,total
purpose,
0,10814
1,4308
2,4014
3,2335


In [72]:
t_8 = d_1.groupby(['purpose'], as_index = False).aggregate({'debt': 'sum'}) \
    .rename(columns = {'debt': 'amount_of_debts'})
t_8['total'] = t_7['total']
t_8['percent'] = t_8['amount_of_debts'] / t_8['total'] * 100
t_8

,purpose,amount_of_debts,total,percent
0,0,782,10814,7.231367
1,1,403,4308,9.354689
2,2,370,4014,9.217738
3,3,186,2335,7.965739


Кредиты, взятые на такие цели, как "автомобиль", "образование", возвращают немного реже

Среди проведенных исследований зависимостей не было найдено показательных связей. 